# Modeling - Sklearn Decision Tree

## Summary
If a simpler model can get results that are as good as a more complex model, it is better to use the simpler one, to maintain interpretability and keep computational costs low. In this notebook, I'm going to spend a little time seeing how good a base decision tree from sklearn can get.

# DT Modeling

In [1]:
# Import statements
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

# Importing metrics function from functions.py
from functions import metrics as custom_score

In [2]:
# Load in cleaned data.

# Training Data
X_train = pd.read_csv('../Data/train/X_train.csv', index_col=0)
y_train = pd.read_csv('../Data/train/y_train.csv', index_col=0)

# Testing Data
X_test = pd.read_csv('../Data/test/X_test.csv', index_col=0)
y_test = pd.read_csv('../Data/test/y_test.csv', index_col=0)

## Base DT Model
Re-creating a base decision tree to compare our models too going forward

In [4]:
# Instantiating Tree
FSM_DT = DecisionTreeClassifier()

# Fitting Model
FSM_DT.fit(X_train, y_train)

# Score on the testing data.
custom_score(y_test, FSM_DT.predict(X_test))

Accuracy: 0.90
Precision: 0.48
Recall: 0.54
F1: 0.51
ROC AUC: 0.74
